# 5. Библиотеки в Julia



## 5.1. Модули и Пакеты

В Julia можно разбивать код программы на модули (*modules*).
Модуль создаёт собственное пространство имён и может быть прекомпилирован.

Основной синтаксис выглядит так.

```julia
module Points
using LinearAlgebra
export dist, Point
include("types.jl")
include("functions.jl")
private_foo() = println("Hello!")
end # module
```

Всё, что между командами `module ... end` представляет собой модуль.
В данном примере создаётся модуль `Points`. 

Инструкция `using LinearAlgebra` импортирует публичные имена из модуля `LinearAlgebra`.
При таком вызове, например, функция `LinearAlgebra.norm` из модуля доступна просто по имени `norm`.
Фактически, программист так указывает зависимости модуля `Points` от других модулей.

Модуль `Points` также делает имена `dist` и `Point` публичными. Т.е., когда кто-нибудь импортирует `Points` командой `using`, то ему будут доступны имена `dist` и `Point`. Точно также где-то в исходном коде модуля `LinearAlgebra` происходит экспорт имени `norm`.

Функция `include("<path_to_file>")` делает подстановку содержимого файла в модуль.
В Julia позднее связывание имён, поэтому вы можете спокойно экспортировать что-то, а объявить где-то позднее.

### Как импортировать свой модуль?

Если модуль `Points` помещён в файл `myscripts/Points.jl`, то можно импортировать модуль так
```julia
include("myscripts/Points.jl")  # В общем случае полный путь до файла с исходным кодом модуля
using .Points

@show dist(Point(1, 2))
```
Здесь функцией `include` совершается подстановка содержимого файла с модулем, как будто бы мы его скопировали сюда.
Затем используется `using .Points` -- заметьте точку.
По умолчанию Julia ищет модули (а точнее, пакет) в текущем окружении.
Глобальное окружение вы можете посмотреть в REPL командой `] status` -- она покажет список установленных пакетов (но не покажет стандартные библиотеки).
В нашем случае пакет не создавался и регистрировался в глобальном окружении, поэтому команда `using Points` привела бы к ошибке.
Но, с помощью `.` поиск модуля осуществляется относительно скрипта, а не окружения.
Кроме того, есть `..` для обращения к родительскому модулю.

### using и import

Для импортирования существуют инструкции `using` и `import`.
Их главное отличие в назначении.

Инструкция `using` предназначена для использования кода модуля *пользователем*.
Например, `using LinearAlgebra` позволяет нам, как пользователям модуля `LinearAlgebra`, использовать функции `norm`, `cross`...

Инструкция `import` отличается от `using` тем, что позволяет переопределять и создавать новые методы для функций, определённых в импортированном модуле.
Другими словами, `import` для разработчиков.
Например, чтобы добавить метод скалярного произведения для собственного типа данных, придётся воспользоваться `import LinearAlgebra`.

* `using Points`:

  - `Points`, и публичные `dist`, `Point`, остальные через точку: `Points.private_foo`

* `using Points: dist`:

  - Только `dist`

* `using Points: dist as d`:

  - Только `d`

* `import Points`:

  - `Points`, остальные через точку

* `import Points as Pnts`:

  - `Pnts`, остальные через точку

### Пример разработки модуля

Ниже показана разработка модуля в несколько этапов. В нём привычная структура `Point{T}`, а её интерфейс оборачивается в модуль. Затем, для примера, структура встраивается в существующую экосистему языка: можно скалярно умножать точки, складывать или умножать на скаляр.

```julia
module Points

import LinearAlgebra  # для добавления метода к скалярному произведению

export dist
export Point

struct Point{T}
    x::T
    y::T
end

dist(p::Point) = sqrt(p.x^2 + p.y^2)
random_point() = Point(rand(2)...)

# Добавление метода к скалярному произведению LinearAlgebra.dot
LinearAlgebra.dot(p1::Point, p2::Point) = p1.x * p2.x + p1.y * p2.y

# Расширяение стандартной библиотеки языка, модуля Base
# `+` коммутативно
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)
# `*` не коммутативно
Base.:*(α::Number, p::Point) = Point(α * p.x, α * p.y)
Base.:*(p::Point, α::Number) = α * p

end # module
```

In [1]:
include("data/Points.jl")
using .Points

println(Point(1, 2) + Point(3.0, 4.1))
println(2 * Point(1, 2))
println(Point(1, 2) * 2.0)

Point{Float64}(4.0, 6.1)
Point{Int64}(2, 4)
Point{Float64}(2.0, 4.0)


### Разбиение исходного кода на файлы

Когда исходный код разрастается, его разбивают на отдельные файлы.
Для библиотеки кода в Julia типично наличие корневого файла, в котором объявлен модуль, зависимости от других библиотек и экспортируемые имена, а оставшийся код библиотеки подставлятся с помощью `include`.
Весь код библиотеки помещают в директорию `src`, а корневой файл называют также, как и главный модуль библиотеки.

От реальной структуры отличается только отстутсвием файла с зависимостями пакета.

Ниже показана типичная структура исходного когда библиотеки.

Структура библиотеки и `Points.jl`

Структура директории:
``` Structure
  src/
   ├─operators.jl
   ├─interface.jl
   ├─types.jl
   └─Points.jl
```

Код модуля `src/Points.jl`:

```julia
module Points
import LinearAlgebra
export dist
export Point
include("types.jl")
include("interface.jl")
include("operators.jl")
end # module
```

`src/types.jl`:
```julia
struct Point{T}
    x::T
    y::T
end
```

`src/interface.jl`:
```julia
dist(p::Point) = sqrt(p.x^2 + p.y^2)
random_point() = Point(rand(2)...)
```

`src/operators.jl`:
```julia
LinearAlgebra.dot(p1::Point, p2::Point) = p1.x * p2.x + p1.y * p2.y
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)
Base.:*(α::Number, p::Point) = Point(α * p.x, α * p.y)
Base.:*(p::Point, α::Number) = α * p
```

### Пакеты


Пакет (*package*) это инструмент для распространения библиотеки кода среди пользователей.
Пакет содержит саму библиотеку кода и метаинформацию о нём, необходимую для идентификации пакета в системе и корректной установки.
Кроме того, пакет может содержать примеры применения и набор тестов.

Пакеты в Julia создаются с помощью Pkg-mode в Julia REPL.
Сам пакет представляет из себя git-репозиторий со следующей структурой

``` Structure
  AwesomeLibrary/
   ├─src/
   │  ├─AwesomeLibrary.jl
   │  └─ ...
   ├─tests/
   │  └─ ...
   └─Project.toml
```

Непосредственный код библиотеки содержится в `src/` и имеет ту же структуру, которую мы рассмотрели в примере разработки модуля.
В `tests/` содержится система тестирования кода библиотеки.
В `Project.toml` содержится информация об авторе пакета, его уникальный номер, а также перечисление программных библиотек с версиями, которые необходимы для работы пакета (зависимостей).

Пакеты можно разрабатывать и использовать локально.
Это лучше всего делать в директории `~/.julia/dev/`.
Также, если вы хотите внести изменения не в свой пакет, то для этого используется `] dev`, а исходный код появится в `~/.julia/dev/`.

Для распространения пакеты публикуются на git-платформе, обычно на GitHub, например, [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl) или [Unitful.jl](https://github.com/PainterQubits/Unitful.jl).
Кроме того, пакеты обычно регистрируются в реестре пакетов для удобства поиска пользователями.
Реестр также является git-репозиторием и хранит информацию о всех зарегистрированных версиях пакетов.
В REPL вы можете посмотреть добавленные реестры командой `] registry status`.
Скорее всего, там окажется `General` реестр -- официальный реестр пакетов для Julia.
Можно искать пакеты в REPL, на github или на интернет-платформе, вроде [такой](https://juliapackages.com/).

Зарегистрированные в реестре пакеты устанавливаются просто: `] add PackageName`, например, `] add DataFrames`.
После этой команды Julia запросит информацию в реестрах о пакете `DataFrames`, найдёт его фактическое расположение, скачает и установит все зависимости.
Список установленных пакетов в окружении можно узнать командой `] status`.

В Julia пакеты имеют независимые окружения.
Т.е. зависимости пакета изолированы от остальных окружений, в том числе от глобального.
Это уменьшает количество конфликтов версий между используемыми библиотеками.

Полная информация об окружении содержится в файле `Project.toml`.
Если вы хотите поделиться кодом, который использует библиотеки, то можно вместе с ним передать `Project.toml` файл.
Так поступают для Jupyter ноутбуков, а в Pluto.jl информация об окружении содержится внутри скрипта.

В окружение также входит файл `Manifest.toml`, создаваемый автоматически.
В нём содержится полная и точная информация о зависимостях пакета для вашего компьютера.
Т.е. какая библиотека откуда берётся.


## 5.2. Линейная алгебра

В Julia богатая стандартная библиотека операций линейной алгебры. Часто используемые инструменты доступны сразу.

Ниже представлены примеры операций для этих векторов и матриц.

In [2]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [1; 2]

A = [1 2; 
     3 4]
B = [-1 -2; 
     -3 -4]
C = [1im 2  ;
     3  -1im]
# Линейные операции:
@show 2*a + b - a / 2
@show A/2 + 2*B

# Матричное умножение
@show A * c
@show A * B

# Транспонирование
@show a'
@show A'

# Для комплексных матриц `'`-оператор выполняет эрмитово сопряжение.
@show C';

# Решение СЛАУ
@show x = A \ [1, 2]

# Обратная матрица
@show inv(A)

(2a + b) - a / 2 = [5.5, 8.0, 10.5]
A / 2 + 2B = [-1.5 -3.0; -4.5 -6.0]
A * c = [5, 11]
A * B = [-7 -10; -15 -22]
a' = [1 2 3]
A' = [1 3; 2 4]
C' = Complex{Int64}[0 - 1im 3 + 0im; 2 + 0im 0 + 1im]
x = A \ [1, 2] = [0.0, 0.5]
inv(A) = [-1.9999999999999996 0.9999999999999998; 1.4999999999999998 -0.4999999999999999]


2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5

### LinearAlgebra

Хотя большая часть линейной алгебры доступна в Julia по умолчанию (как показано выше), есть стандартная библиотека с именем `LinearAlgebra`, которая включает в себя гораздо больше релевантных имен и функций. В частности, он предоставляет факторизации и некоторые типы структурированных матриц. Как и во всех пакетах, вы можете использовать эти дополнительные функции в своем сеансе с помощью `using LinearAlgebra`.

In [3]:
using LinearAlgebra
# стандартные функции
@show det(A) # детерминант
@show tr(A) # след
@show norm(a) # норма

# скалярное умножение
@show dot(a, b)
@show a ⋅ b #\cdot + TAB
@show A ⋅ B 

# векторное умножение
@show cross(a, b)
@show a × b  # \times + TAB

# cобственные значения
@show eigvals(A)

A = rand(3, 3)
x = fill(1, (3,))
b = A * x;

det(A) = -2.0
tr(A) = 5
norm(a) = 3.7416573867739413
dot(a, b) = 32
a ⋅ b = 32
A ⋅ B = -30
cross(a, b) = [-3, 6, -3]
a × b = [-3, 6, -3]
eigvals(A) = [-0.3722813232690143, 5.372281323269014]


### Факторизация

**LU-факторизация**
```julia
PA = LU
``` 
где `P` матрица перестановок, `L` нижняя треугольная матрица и `U` - верхняя. Используется `lufact`.

Julia позволяет вычислять  LU-факторизацию и определяет составной тип факторизации для его хранения.

In [4]:
Alu = lu(A)
@show typeof(Alu)
@show Alu.P
@show Alu.L
@show Alu.U;

@show A\b
@show Alu\b
@show det(A) ≈ det(Alu);

typeof(Alu) = LU{Float64, Matrix{Float64}, Vector{Int64}}
Alu.P = [0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 0.0 1.0]
Alu.L = [1.0 0.0 0.0; 0.11593147828135314 1.0 0.0; 0.9556929327550823 0.3532152648968352 1.0]
Alu.U = [0.8624309894297549 0.3554984851454187 0.20579039458482895; 0.0 0.7331121005518092 0.39780727910539515; 0.0 0.0 0.11689820227739639]
A \ b = [0.9999999999999999, 1.0000000000000002, 0.9999999999999996]
Alu \ b = [0.9999999999999999, 1.0000000000000002, 0.9999999999999996]
det(A) ≈ det(Alu) = true


**QR-факторизация**
```
A=QR
``` 

где `Q` унитарная/ортогональная матрица и `R` верхнетреугольная матрица. Используется `qrfact`. 

In [5]:
Aqr = qr(A)
@show Aqr.Q
@show Aqr.R;

Aqr.Q = LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}([-1.1971303538002684 -0.7329754254808017 -0.4961051916530409; 0.6648848797296597 0.7396894724942118 0.4162386518719645; 0.6354257806533486 0.25900982723469484 0.08346640213151071], [1.0835188074572226 -1.6844797322001315 0.0; 6.9359543723792e-310 1.8742630211473104 0.0; 6.9359543723792e-310 6.93595540394794e-310 0.0])
Aqr.R = [-1.1971303538002684 -0.7329754254808017 -0.4961051916530409; 0.0 0.7396894724942118 0.4162386518719645; 0.0 0.0 0.08346640213151071]


**Спектральное разложение матрицы**

Результаты собственных разложений, разложения по сингулярным числам, разложения по Гессенбергу и разложения Шура хранятся в типах `Factorization`. 

Собственная декомпозиция может быть вычислена как:

In [6]:
Asym = A + A'
AsymEig = eigen(Asym)

#Значения и векторы могут быть извлечены из типа Eigen:
@show AsymEig.values
@show AsymEig.vectors;

AsymEig.values = [-1.311103656125741, 0.06363484068534975, 3.066596047727915]
AsymEig.vectors = [0.798830355099682 -0.15600955427680913 -0.5809742530832732; -0.5470825983111212 -0.5900072020532066 -0.5937946885502589; -0.25014134883395095 0.7921821258395685 -0.5566657750426496]


Еще раз, когда факторизация хранится в типе, мы можем отправить его и написать специальные методы, которые используют свойства факторизации, например, $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [7]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          -4.44089e-15  9.10383e-15
 -1.95399e-14   1.0          4.17444e-14
  2.30926e-14   2.04281e-14  1.0

### Специальные матричные структуры

Мы можем объявить структуру явно с помощью, например, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` и `SymTridiagonal`.

In [8]:
a = [1, 10, 100]
B = [ 1.5  2 -4; 
      2.0 -1 -3; 
     -4.0 -3  5]
     
@show Diagonal(a)
@show Symmetric(B)
@show Hermitian(B)
@show UpperTriangular(B)
@show LowerTriangular(B);


Diagonal(a) = [1 0 0; 0 10 0; 0 0 100]
Symmetric(B) = [1.5 2.0 -4.0; 2.0 -1.0 -3.0; -4.0 -3.0 5.0]
Hermitian(B) = [1.5 2.0 -4.0; 2.0 -1.0 -3.0; -4.0 -3.0 5.0]
UpperTriangular(B) = [1.5 2.0 -4.0; 0.0 -1.0 -3.0; 0.0 0.0 5.0]
LowerTriangular(B) = [1.5 0.0 0.0; 2.0 -1.0 0.0; -4.0 -3.0 5.0]


## 5.3. Как обработать лабу

Короткая демонстрация потенциальных возможностей графики в Julia:

In [1]:
using Plots
# define the Lorenz attractor
Base.@kwdef mutable struct Lorenz
    dt::Float64 = 0.02
    σ::Float64 = 10
    ρ::Float64 = 28
    β::Float64 = 8/3
    x::Float64 = 1
    y::Float64 = 1
    z::Float64 = 1
end

function step!(l::Lorenz)
    dx = l.σ * (l.y - l.x)
    dy = l.x * (l.ρ - l.z) - l.y
    dz = l.x * l.y - l.β * l.z
    l.x += l.dt * dx
    l.y += l.dt * dy
    l.z += l.dt * dz
end

attractor = Lorenz()


# initialize a 3D plot with 1 empty series
plt = plot3d(
    1,
    xlim = (-30, 30),
    ylim = (-30, 30),
    zlim = (0, 60),
    title = "Lorenz Attractor",
    legend = false,
    marker = 2,
)

# build an animated gif by pushing new points to the plot, saving every 10th frame
@gif for i=1:1500
    step!(attractor)
    push!(plt, attractor.x, attractor.y, attractor.z)
end every 10;

┌ Info: Saved animation to /tmp/jl_g5a18ZW9Mz.gif
└ @ Plots /home/helga/.julia/packages/Plots/ju9dp/src/animation.jl:156


### Plots

Есть несколько разных способов построения графика в Julia (включая вызов PyPlot). 

Здесь мы рассмотрим, как использовать `Plots.jl`. Если он еще не установлен, вам нужно использовать менеджер пакетов для его установки, и Julia прекомпилирует его для вас при первом использовании:

In [4]:
#using Pkg
#Pkg.add("Plots")
#Pkg.add("Plotly")
using Plots
using LaTeXStrings
using LsqFit

globaltemperatures = [14.4, 14.5, 14.8, 15.2, 15.5, 15.8]
numpirates = [45000, 20000, 15000, 5000, 400, 17];

# Фиттирование функцией
exp_pir(x, p) = p[1] .* x.^(-1) .+ p[2] 
p0 = [14.0, 2.0e4]
fitt = curve_fit(exp_pir, numpirates, globaltemperatures, p0)
A, κ = fitt.param
dA, dκ = margin_error(fitt)
Exp(x) = exp_pir(x, fitt.param)

# Режим отображения
# Для отображения в терминале есть специальный режим:
# using UnicodePlots
# unicodeplots()

plotly() # можно попробовать gr()

Plots.PlotlyBackend()

Одним из преимуществ `Plots.jl` является то, что он позволяет вам легко менять серверные части. В этом блокноте мы опробуем бэкенды `plotly()` и `plotlyjs ()`. 

Во имя научного исследования давайте воспользуемся этой записной книжкой для изучения взаимосвязи между глобальной температурой и числом пиратов между примерно 1860 и 2000 годами.

In [5]:
# Отрисовка линии на графике
plot(numpirates, globaltemperatures, label="line", lc=:blue, ls=:dash) 

# Отрисовка функции на графике
plot!(numpirates->Exp(numpirates); seriestype=:path, lc=:black, ls=:dash, label="spline") 

# Отрисовка точек на графике
scatter!(numpirates, globaltemperatures, label="points", mc=:green, ms=5, ma=10, lc=:black) 

# Латеховская строка по литералу
ylabel!(L"Global Temperature (C $\circ$)")
xlabel!("Number of Pirates [Approximate]")
title!("Influence of pirate population on global warming")

### Unitful

Ещё одна очень полезная библиотека для физиков - это Unitful и различные её расширения (например, UnitfulAstro). Библиотека добавляет размерные единицы измерений, которые задаются через строковые литералы u"..."

In [13]:
using Unitful
using UnitfulAstro
using LinearAlgebra

m = 250.5u"g"
V = 12.6u"cm/s"
g = 9.8u"m/s^2"
h = 10.0u"m"
K = m * V^2 / 2
P = m * g * h

# Приведение единиц измерения (к мДж)
@show K |> u"mJ"

# Приведение к безразмерной величине
@show K/P |> NoUnits

# Проверка размерности
@show dimension(K)

# Логарифмические единицы
@show uconvert(u"mW*s", 20u"dBm/Hz")

# Степенные величины
@show 1.0u"V/sqrt(Hz)"

# Различные варианты конвертации
@show uconvert(u"hr",3602u"s")
@show u"cm"(1u"m")
@show 1u"m" |> u"cm"

# Приведение к общему типу
@show promote(1.0u"J/K", 1.0u"g*cm^2/s^2/K")

# Перевод в СИ
@show upreferred(u"N*m")

# Приведенная постоянная Планка
@show u"ħ"

# Астрономические единицы
@show K |> u"erg"
au = 1.0u"AU"
@show au |> u"cm"
m_ns = 1.5u"Msun"
@show m_ns |> u"g";

K |> u"mJ" = 1.988469 mJ
K / P |> NoUnits = 8.1e-5
dimension(K) = 𝐋^2 𝐌 𝐓^-2
uconvert(u"mW*s", 20 * u"dBm/Hz") = 100.0 s mW
1.0 * u"V/sqrt(Hz)" = 1.0 V Hz^-1/2
uconvert(u"hr", 3602 * u"s") = 1801//1800 hr
(u"cm")(1 * u"m") = 100 cm
1 * u"m" |> u"cm" = 100 cm
promote(1.0 * u"J/K", 1.0 * u"g*cm^2/s^2/K") = (1.0 kg m^2 K^-1 s^-2, 1.0e-7 kg m^2 K^-1 s^-2)
upreferred(u"N*m") = kg m^2 s^-2
u"ħ" = 1.0545718176461565e-34 J s
K |> u"erg" = 19884.69 erg
au |> u"cm" = 1.495978707e13 cm
m_ns |> u"g" = 2.9826148060470766e33 g
